## Parameters

In [ ]:
endpoint_url = 'http://rook-ceph-rgw-s3a.openshift-storage.svc.cluster.local'
aws_access_key_id = 'replace_me'
aws_secret_access_key = 'replace_me'
bucket_base_name = 'replace_me'
namespace = 'replace_me'

## Imports

In [ ]:
import boto3
import json
import botocore
import argparse

## S3 and SNS connections

In [ ]:
s3 = boto3.client('s3',
                endpoint_url = endpoint_url,
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                region_name = 'default',
                config=botocore.client.Config(signature_version = 's3'))

sns = boto3.client('sns', 
                endpoint_url = endpoint_url, 
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key= aws_secret_access_key,
                region_name='default', 
                config=botocore.client.Config(signature_version = 's3'))

## Create buckets

In [ ]:
def create_bucket(bucket_name):
    result = s3.create_bucket(Bucket=bucket_name)
    return result

In [ ]:
create_bucket(bucket_base_name)
create_bucket(bucket_base_name+'-processed')
create_bucket(bucket_base_name+'-anonymized')

### Verification

In [ ]:
for bucket in s3.list_buckets()['Buckets']:
    print(bucket['Name'])

In [ ]:
# Make bucket public read
for bucket in s3.list_buckets()['Buckets']:
    bucket_policy = {
                      "Version":"2012-10-17",
                      "Statement":[
                        {
                          "Sid":"AddPerm",
                          "Effect":"Allow",
                          "Principal": "*",
                          "Action":["s3:GetObject"],
                          "Resource":["arn:aws:s3:::{0}/*".format(bucket['Name'])]
                        }
                      ]
                    }
    bucket_policy = json.dumps(bucket_policy)
    s3.put_bucket_policy(Bucket=bucket['Name'], Policy=bucket_policy)

## Bucket Notifications configuration

In [ ]:
attributes = {}
attributes['push-endpoint'] = 'kafka://my-cluster-kafka-bootstrap.'+namespace+':9092'
attributes['kafka-ack-level'] = 'broker'

In [ ]:
def create_topic(topic):
    topic_arn = sns.create_topic(Name=topic, Attributes=attributes)['TopicArn']
    return topic_arn

In [ ]:
create_topic('xray-images')

In [ ]:
sns.list_topics()

In [ ]:
bucket_notifications_configuration = {
            "TopicConfigurations": [
                {
                    "Id": 'xray-images',
                    "TopicArn": 'arn:aws:sns:s3a::xray-images',
                    "Events": ["s3:ObjectCreated:*"]
                }
            ]
        }

s3.put_bucket_notification_configuration(Bucket = bucket_base_name,
        NotificationConfiguration=bucket_notifications_configuration)

In [ ]:
s3.get_bucket_notification_configuration(Bucket = bucket_base_name)

In [ ]:
sns.delete_topic(TopicArn='arn:aws:sns:ocs-storagecluster-cephobjectstore::xray-images')